### Welcome to the tennis predictor

Instructions
Please modify the list of players following the format, modify the match and then go to a markdown cell titled "RUN".

In [12]:
# Add players or modify form
Sinner = {
    "name" : "Sinner",
    "form" : 95
}

Alcaraz = {
    "name" : "Alcaraz",
    "form" : 100
}

Djokovic = {
    "name" : "Djokovic",
    "form" : 85
}

# Modify game settings, like players, number of sets, presence of advantage points etc
match(player_1 = Sinner, player_2 = Alcaraz, num_sets_to_win = 3, tie_break_points = 7, advantage_tie_break = True)

Sinner has a 33.32% chance while Alcaraz has a 66.68% chance of winning.


### RUN

Now run the cell below to get the logic running and then run the cell above to get your predictions!

In [ ]:
import numpy as np
import math

def deuce(p1):
    prob_1_wins_deuce = p1 ** 2 /(p1 ** 2 + (1  -p1) ** 2)
    return {
        "prob 1 wins deuce" : prob_1_wins_deuce,
        "prob 2 wins deuce" : 1 - prob_1_wins_deuce
    }

def clash(p1, points_to_reach, advantage = True):
    p1_deuce = deuce(p1)["prob 1 wins deuce"]
    prob_1_wins = 0
    for i in range(points_to_reach):
        prob_1_wins += p1 ** points_to_reach * (1 - p1) ** i * math.comb(points_to_reach + i - 1, i)
    if advantage == True:
        prob_1_wins = prob_1_wins - p1 ** points_to_reach * (1 - p1) ** (points_to_reach - 1) * math.comb(2 * points_to_reach - 2, points_to_reach - 1) + p1 ** (points_to_reach - 1) * (1 - p1) ** (points_to_reach - 1) * math.comb(2 * points_to_reach - 2, points_to_reach - 1) * p1_deuce
    return {
        "prob 1 wins clash" : prob_1_wins,
        "prob 2 wins clash" : 1 - prob_1_wins
    }

def game(p1):
    prob_1_wins_game = clash(p1, 4, advantage=True)["prob 1 wins clash"]
    return {
        "prob 1 wins game" : prob_1_wins_game,
        "prob 2 wins game" : 1 - prob_1_wins_game
    }

def tie_break(p1, points_to_reach = 7, advantage = True):
    prob_1_wins_tie_break = clash(p1, points_to_reach, advantage)["prob 1 wins clash"]
    return {
        "prob 1 wins tie break" : prob_1_wins_tie_break,
        "prob 2 wins tie break" : 1 - prob_1_wins_tie_break
    }


def set(prob_1_wins_point, tie_break_points, advantage_tie_break = True):
    p1 = game(prob_1_wins_point)["prob 1 wins game"]
    p2 = 1 - p1

    p1_tie = tie_break(prob_1_wins_point, tie_break_points, advantage_tie_break)["prob 1 wins tie break"]
    p2_tie = 1 - p1_tie

    start = np.array([1] + [0]*40)
    matrix = np.zeros((41, 41), dtype=np.float64)

    for i in range(30):
        matrix[i, i + 6] = p1
    for i in range(35):
        if (i+1) % 6 == 0:
            matrix[i, 40] = p2
        else:
            matrix[i, i + 1] = p2
    for i in range(5):
        matrix[30 + i, 39] = p1
    matrix[35, 36] = p1
    matrix[37, 38] = p1
    matrix[36, 39] = p1
    matrix[35, 37] = p2
    matrix[36, 38] = p2
    matrix[37, 40] = p2
    matrix[38, 39] = p1_tie
    matrix[38, 40] = p2_tie
    matrix[39, 39] = 1
    matrix[40, 40] = 1

    lim_transition = np.linalg.matrix_power(matrix, 10000)

    terminal = start @ lim_transition
    prob_1_wins_set = terminal[39]
    prob_2_wins_set = 1 - prob_1_wins_set

    return {
        "prob 1 wins set" : prob_1_wins_set,
        "prob 2 wins set" : prob_2_wins_set
    }

def match(player_1, player_2, num_sets_to_win = 3, tie_break_points = 7, advantage_tie_break = True):
    
    player_1["probability"] = player_1["form"] / (player_1["form"] + player_2["form"])
    player_2["probability"] = 1 - player_1["probability"]

    prob_1_wins_set = set(player_1["probability"], tie_break_points, advantage_tie_break)["prob 1 wins set"]

    prob_1_wins_match = clash(prob_1_wins_set, num_sets_to_win, advantage = False)["prob 1 wins clash"]
    
    prob_2_wins_match = 1 - prob_1_wins_match

    prob_1 = round(100 * prob_1_wins_match, 2)
    prob_2 = round(100 * prob_2_wins_match, 2)
    
    print(f"{player_1['name']} has a {prob_1}% chance while {player_2['name']} has a {prob_2}% chance of winning.")